# Problem c

In [3]:
from scipy.io import loadmat
import numpy as np
import random
news = loadmat('news.mat')

In [4]:
data = news['data'].toarray()
labels = news['labels'].flatten()
testdata = news['testdata'].toarray()
testlabels = news['testlabels'].flatten()

In [5]:
ind = np.array(range(len(labels)))
nseq1 = ind[np.logical_or(labels ==1, np.logical_or(labels == 16, labels == 20))]
pseq2 = ind[np.logical_or(labels ==17, np.logical_or(labels == 18, labels == 19))]
labels = np.append(np.array([0]*nseq1),1 + np.array([0]*pseq2))
data = np.append(data[nseq1],data[pseq2], axis=0)
ind = np.array(range(len(testlabels)))
nseq1 = ind[np.logical_or(testlabels ==1, np.logical_or(testlabels == 16, testlabels == 20))]
pseq2 = ind[np.logical_or(testlabels ==17, np.logical_or(testlabels == 18, testlabels == 19))]
testlabels = np.append(np.array([0]*nseq1),1 + np.array([0]*pseq2))
testdata = np.append(testdata[nseq1],testdata[pseq2], axis=0)

In [6]:
def NBprior(labels,i):
    ind = np.array(range(len(labels)))
    seq = ind[labels == i]
    return seq

In [7]:
def NBtrain(data,labels):
    if data.shape[0] == len(labels):
        d = data.shape[1]
        pi = np.zeros(2)
        mu = np.zeros((d,2))
        for y in range(2):
            traindata = data[NBprior(labels,y)]
            l = traindata.shape[0]
            unit = np.array([1]*l) 
            pi[y] = l/len(labels)
            mu[:,y] = (1 + np.dot(unit,traindata))/(2 + l)
        return pi,mu
    else:
        raise Exception('invalid input')

In [8]:
nbtrain = NBtrain(data,labels)
pi = nbtrain[0][np.newaxis]
mu = nbtrain[1]

In [9]:
def errorate(pi,mu,testdata,testlabels):
    M = np.log(pi)+np.dot(testdata,np.log(mu))+np.dot(1-testdata,np.log(1-mu))
    preds = np.argmax(M, axis=1)
    if len(preds)==len(testlabels):
        return (np.count_nonzero(preds-testlabels)/len(testlabels))
    else:
        raise Exception('invalid input')

In [10]:
import datetime
print (datetime.datetime.now())
trainpreds = errorate(pi,mu,data,labels)
testerror = errorate(pi,mu,testdata,testlabels)
print (datetime.datetime.now())

2016-09-27 13:51:42.340757
2016-09-27 13:51:43.661820


In [11]:
print(trainpreds)
print(testerror)

0.05779392338177015
0.13138324243926625


# Problem d

In [45]:
#alpha = np.append(sum(np.log((1-mu[:,1])/(1-mu[:,0])))+np.log(pi.flatten()[1]/pi.flatten()[0]),np.log((mu[:,1]*(1-mu[:,0]))/(mu[:,0]*(1-mu[:,1]))))

In [12]:
alpha = np.log((mu[:,1]*(1-mu[:,0]))/(mu[:,0]*(1-mu[:,1])))

In [27]:
f = open('news.vocab')
words = [line.rstrip('\n') for line in open('news.vocab')]

In [28]:
smallest = np.array(words)[alpha.argsort()[:20].flatten()]
largest = np.array(words)[alpha.argsort()[-20:].flatten()]
print(smallest)#from small to large 
print(largest)#from small to large

['athos' 'atheism' 'atheists' 'clh' 'teachings' 'revelation' 'testament'
 'livesey' 'atheist' 'solntze' 'wpd' 'scriptures' 'theology' 'believers'
 'ksand' 'alink' 'benedikt' 'jesus' 'prophet' 'mozumder']
['arabs' 'budget' 'firearm' 'palestine' 'optilink' 'handgun' 'sdpa' 'cpr'
 'armenia' 'cramer' 'villages' 'sahak' 'melkonian' 'ohanus' 'appressian'
 'argic' 'serdar' 'israelis' 'occupied' 'firearms']
